<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTunning_Testing_For_AviationQADataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/Cloud_curious/blob/master/FineTuning_Mistral_7b_hfdeployment_dataset_AviationQA.ipynb

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

!pip install -U tensorboard-plugin-profile -q

In [ ]:

#!pip install colab-env --quiet

!pip install -U tensorboard-plugin-profile -q

import colab_env


13_21-27-00_5405048c77e4

0.0167 min loss
1.2073
1.2073

670 last step

1.1882
-98%

10
0.0191 last loss

Jun14_00-31-36_f18dc487e267

0.0069
1.1853
1.1853
0.0107

1.1746
-99%

10
1,150


Jun14_00-31-36_f18dc487e267

0.0066
1.1853
1.1853
0.0135

1.1718
-99%

10
1,400

In [ ]:
# Load the TensorBoard notebook extension

%load_ext tensorboard
%tensorboard --logdir /content/gdrive/MyDrive/model/Mistral-7B-v0.1_AviationQA/runs

In [4]:
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [5]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import torch
import torch.nn as nn
from huggingface_hub import PyTorchModelHubMixin

class MyModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, num_channels: int, hidden_size: int, num_classes: int):
        super().__init__()
        self.param = nn.Parameter(torch.rand(num_channels, hidden_size))
        self.linear = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        return self.linear(x + self.param)

# create model
#config = {"num_channels": 3, "hidden_size": 32, "num_classes": 10}
#model = MyModel(**config)

# save locally
#model.save_pretrained("my-awesome-model")

# push to the hub
#model.push_to_hub("my-awesome-model")

# reload
#model = MyModel.from_pretrained("username/my-awesome-model")

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Mistral-7B-v0.1_AviationQA"
#peft_model_id = '/content/gdrive/MyDrive/model/Mistral-7B-v0.1_AviationQA'


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)


# save locally
#model.save_pretrained("Mistral-7B-v0.1_AviationQA-40")

# push to the hub
#model.push_to_hub("Mistral-7B-v0.1_AviationQA-40epoch")


# reload
#model = MyModel.from_pretrained("username/my-awesome-model")

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset AviationQA")
dataset = load_dataset("sakharamg/AviationQA")

# save datasets to disk
dataset["train"].to_json("train_dataset_AviationQA.json", orient="records")
dataset["validation"].to_json("validation_dataset_AviationQA.json", orient="records")
dataset["test"].to_json("test_dataset_AviationQA.json", orient="records")

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [10]:
dataset

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 10807
})

In [11]:
nrec=10
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['Question'][0:nrec]
dataset_final_Answer=dataset['Answer'][0:nrec]

In [12]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [13]:
datasetF

,id,Question,Answer
0,a4905077116,What is the name of the Engine Manufacturer of...,Lycoming
1,a7510929859,When was the Last Flight Review or Equivalent ...,"May 13, 2017"
2,a5430892651,What was the flight conducted under for the ac...,Part 91: General aviation - Personal
3,a9405153070,What was the Observation Time of the accident ...,13:54 Local
4,a8193000066,What is the name of the Engine Manufacturer of...,LYCOMING
5,a9610719319,What is the Airworthiness Certificate of the a...,Normal
6,a7001234003,Was there any Operating Certificate(s) held of...,None
7,a9135502144,How many Seats are there in the aircraft beari...,2
8,a157434574,What is the Serial Number of the aircraft bear...,07-12-03
9,a1523521076,What is the Airframe Total Time of the acciden...,10663 Hrs as of last inspection


In [14]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [15]:
eval_dataset

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 10807
})

In [33]:
nrec=2
eval_dataset[nrec]

{'id': 'a5430892651',
 'Question': 'What was the flight conducted under for the accident no. GAA18CA423?',
 'Answer': 'Part 91: General aviation - Personal'}

In [34]:
eval_dataset[nrec]['Question']

'What was the flight conducted under for the accident no. GAA18CA423?'

In [35]:
eval_dataset[nrec]['Answer']

'Part 91: General aviation - Personal'

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [36]:
from random import randint
rand_idx = randint(0, len(eval_dataset))

rand_idx = 2
#,3600, 5570,10077 ## good answer

In [37]:
rand_idx

2

In [ ]:
# Inspect the structure of the messages
#print(eval_dataset[rand_idx]["messages"][:2])


modified_conversation = [
    {"role": "user", "content": eval_dataset[rand_idx]['Question'] if eval_dataset[rand_idx]['Question'] is not None else ""}, # Handle potential None values
    {"role": "assistant", "content": eval_dataset[rand_idx]['Answer'] if eval_dataset[rand_idx]['Answer'] is not None else ""}  # Handle potential None values
]

#print(modified_conversation)

#Use the chat template to format the input for the language model
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt = eval_dataset[rand_idx]['Question']
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [39]:
outputs

[{'generated_text': 'What was the flight conducted under for the accident no. GAA18CA423? [/INST] Part 91: General aviation - Personal '}]

In [40]:
#rand_idx = randint(0, len(eval_dataset))
print(f"Question:\n{eval_dataset[rand_idx]['Question']}")
print()
print(f"Original Answer:\n{eval_dataset[rand_idx]['Answer']}")
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

Question:
What was the flight conducted under for the accident no. GAA18CA423?

Original Answer:
Part 91: General aviation - Personal

Generated Answer:
Part 91: General aviation - Personal


In [41]:
prompt='capital of russia?'

outputs = generation_pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7,
                              top_k=50, top_p=0.9, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

In [42]:
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

Generated Answer:
Not possible to get or use


In [26]:
q=eval_dataset[1]['Question']

In [27]:
#eval_dataset.shuffle().select(range(number_of_eval_samples))[9]

In [43]:
len(eval_dataset)

10807

In [ ]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

def evaluate(sample):

    prompt = sample['Question']
    #print(prompt)

    #prompt = eval_dataset[rand_idx]['Question']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    #prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    #outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt)+8:].strip()

    oanswer = sample['Answer']

    if oanswer == None:
       oanswer = 'Not possible to get or use'


    #if predicted_answer ==

    #s.upper()

    predicted_answer=predicted_answer.replace("LYCOMING", "Lycoming")

    ansid=2

    if predicted_answer == oanswer:
       if ansid == 1:
            print()
            print('Success')
            print(f"Question:\n{sample['Question']}")
            print(f"Original Answer:\n{sample['Answer']}")
            print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")
       return 1
    else:
       if ansid == 0:
            print()
            print('Fail')
            print(f"Question:\n{sample['Question']}")
            print(f"Original Answer:\n{sample['Answer']}")
            print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")
       return 0

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict

for n in range(number_of_eval_samples):
    #print(n)
    s=eval_dataset[n]
    success_rate.append(evaluate(s))
#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
#    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

Accuracy (Eval dataset and predict) for a sample of 100: 33.00%; nepoch=1

Accuracy (Eval dataset and predict) for a sample of 109: 32.11%
nepoch=32

In [48]:
print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

Accuracy (Eval dataset and predict) for a sample of 1000: 31.00%
